# Entity matching

My baseline will be comparing embeddings of ORG chunks and company names.

The large spacy model already provides embedding functionality, as well as comparison.

In [3]:
import spacy

nlp = spacy.load("ru_core_news_lg")
text = "Didi рассматривает возможность перехода в частную собственность, чтобы успокоить Пекин"
doc = nlp(text)

In [4]:
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Didi 0 4 ORG
Пекин 81 86 LOC
